In [ ]:
# remove this
import sys
import math
import numpy as np

#sys.path.append("/Users/harrison37/Work/alpine/ascent/install-debug/python-modules/")
sys.path.append("/Users/larsen30/research/test_builds/release/ascent/install/python-modules")

# Ascent imports
import conduit
import conduit.blueprint
import ascent

In [ ]:
# cleanup any old results
!./cleanup.sh

In [ ]:
def create_mesh(time):

    mesh1 = conduit.Node()
    xy_dims = 40
    z_dims = 2
    
    conduit.blueprint.mesh.examples.braid("hexs",
                                          xy_dims,
                                          xy_dims,
                                          z_dims,
                                          mesh1)

    field = mesh1["fields/gyre"]
    field["association"] = "vertex"
    field["topology"] = "mesh"
    # https://shaddenlab.berkeley.edu/uploads/LCS-tutorial/examples.html
    values = np.zeros(xy_dims*xy_dims*z_dims)


    e = 0.25
    A = 0.1
    w = (2.0 * math.pi) / 10.0
    a_t = e * math.sin(w * time)
    b_t = 1.0 - 2 * e * math.sin(w * time)
    #print("e: " + str(e) + " A " + str(A) + " w " + str(w) + " a_t " + str(a_t) + " b_t " + str(b_t))
    #print(b_t)
    #print(w)
    idx = 0
    for z in range(z_dims):
        for y in range(xy_dims):
            # scale y to 0-1
            y_n = float(y)/float(xy_dims)
            y_t = math.sin(math.pi * y_n)
            for x in range(xy_dims):
                # scale x to 0-1
                x_f = float(x)/ (float(xy_dims) * .5)
                f_t = a_t * x_f * x_f + b_t * x_f
                #print(f_t)
                value = A * math.sin(math.pi * f_t) * y_t
                u = -math.pi * A * math.sin(math.pi * f_t) * math.cos(math.pi * y_n)
                df_dx = 2.0 * a_t + b_t
                #print("df_dx " + str(df_dx))
                v = math.pi * A * math.cos(math.pi * f_t) * math.sin(math.pi * y_n) * df_dx
                values[idx] = math.sqrt(u * u + v * v)
                #values[idx] = u * u + v * v
                #values[idx] = value
                #print("u " + str(u) + " v " + str(v) + " mag " + str(math.sqrt(u * u + v * v)))
                idx = idx + 1

    #print(values)
    field["values"] = values  
    return mesh1


In [ ]:
# Jupyter imports
from IPython.display import Image
# helper we use when displaying results in the notebook
img_display_width = 500

# open ascent
a = ascent.Ascent()
a.open()

# Setup actions to tell Ascent what to do
# add our expressions definiton to the actions
actions = conduit.Node()

# declare a question to ask 
add_queries = actions.append()
add_queries["action"] = "add_queries"

queries = add_queries["queries"] 
queries["q1/params/expression"] = "entropy(histogram(field('gyre'), num_bins=128))"
queries["q1/params/name"] = "entropy"

# declare triggers 
add_triggers = actions.append()
add_triggers["action"] = "add_triggers"
triggers = add_triggers["triggers"] 

# simple trigger that fires at cycle 500
triggers["t1/params/condition"] = "cycle() == 500"
triggers["t1/params/actions_file"] = "cycle_trigger_actions.yaml"

# a trigger that fires when the change in entroy exceeds 0.5

# the history function allows you to access query results of previous
# cycles. relative_index indicates how far back in history to look.

# Looking at the plot of entropy in the previous notebook, we see a jump
# in entropy at cycle 200, so we expect the trigger to fire at cycle 200
triggers["t2/params/condition"] = "entropy - history(entropy, relative_index = 1) > 0.5"
triggers["t2/params/actions_file"] = "entropy_trigger_actions.yaml"

# view our full actions tree
print(actions.to_yaml())

nsteps = 10
time = 0.0
delta_time = 0.5

info = conduit.Node()
for step in range(nsteps):
    
    # create example mesh using conduit blueprint
    mesh = create_mesh(time)
    
    # update the examples default cycle
    mesh["state/cycle"] = 100 + step * 100
    time = time + delta_time
    
    # publish mesh to ascent
    a.publish(mesh)
    
    # execute
    a.execute(actions)

# close ascent
a.close()

In [ ]:
! ls cycle_trigger*.png

In [ ]:
! ls entropy_trigger*.png

In [ ]:
a.info(info)
print(info["expressions"].to_yaml())